In [1]:
from pathlib import Path

import skimage
from skimage import io
from skimage.transform import resize
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from skimage.filters import laplace

%matplotlib inline

In [6]:
def laplace_multiplicand(fileID):
    fires = io.imread(fileID);
    firstChannel = np.squeeze(fires[:,:,0])
    height, width = firstChannel.shape
    smallFires = resize(firstChannel, (round(height/2), round(width/2)))*255
    blurryFires = resize(smallFires, (height, width))
    details = firstChannel - blurryFires
    laplace_edges = laplace(blurryFires)
    
    table = []
    multiplicand = 0
    while multiplicand <= 1:
        sharped_image = blurryFires+laplace_edges * multiplicand
        abs_error = abs(firstChannel-sharped_image)
        mean = abs_error.mean()
        sqrt = np.sqrt(((details- laplace_edges * multiplicand)**2).mean())
        table.append({'Filename': fileID.name,'Multiplicand': multiplicand, 'Mean Error': mean, 'Square Root': sqrt})
        multiplicand = multiplicand + .1
    return pd.DataFrame(table)


In [8]:
laplace_multiplicand(Path('TIFF_Files/CampFire/GOES-17_ABI_RadC_C07_20181115_170218_cali.tif'))

,Filename,Multiplicand,Mean Error,Square Root
0,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.0,1.699231,2.378094
1,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.1,1.649318,2.294962
2,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.2,1.607010,2.224321
3,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.3,1.573478,2.167394
4,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.4,1.549572,2.125283
5,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.5,1.535613,2.098881
6,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.6,1.531609,2.088781
7,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.7,1.537398,2.095222
8,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.8,1.552775,2.118050
9,GOES-17_ABI_RadC_C07_20181115_170218_cali.tif,0.9,1.576942,2.156747


In [4]:
all_dfs = pd.concat((laplace_multiplicand(file) for file in Path('TIFF_Files/CampFire/').iterdir()))

In [5]:
all_dfs.head()

,Filename,Multiplicand,Mean Error,Square Root
0,TIFF_Files/CampFire/GOES-17_ABI_RadC_C14_20181...,0.0,2.015468,3.054495
1,TIFF_Files/CampFire/GOES-17_ABI_RadC_C14_20181...,0.1,1.953943,2.941140
2,TIFF_Files/CampFire/GOES-17_ABI_RadC_C14_20181...,0.2,1.902410,2.846178
3,TIFF_Files/CampFire/GOES-17_ABI_RadC_C14_20181...,0.3,1.862582,2.771503
4,TIFF_Files/CampFire/GOES-17_ABI_RadC_C14_20181...,0.4,1.835664,2.718786
